## Google Dance

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup 
import requests
import time
import pandas as pd
import pprint
import webbrowser
import urllib
import re
import pymongo
from geopy.geocoders import Nominatim  

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#browser = Browser()

In [3]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=Y5HcXZW5CdG05gKT7KzIAw&q=kids+dance+classes+new+jersey&oq=kids+dance+classes+new+jersey&gs_l=psy-ab.3...140911.143275.0.143677.6.6.0.0.0.0.271.904.0j4j1.5.0....0...1c.1.64.psy-ab..1.0.0....0.2IUp-hMGy8U#rlfi=hd:;si:;mv:[[41.0322952,-73.93508229999999],[40.534548400000006,-74.637359]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["Name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'Name': 'Hit The Barre'}, {'Name': 'Arya Dance Academy'}, {'Name': 'Wild Child Dance Studio'}, {'Name': 'Hudson Dance Studio Edgewater'}, {'Name': 'Class Act Performing Arts Studio'}, {'Name': "Sharon's Studio of Dance & Music"}, {'Name': 'Grooves Unlimited Dance Studio'}, {'Name': 'Dance Palooza Studios'}, {'Name': 'Village Dance Academy'}, {'Name': 'New Jersey Ballet Company and School'}, {'Name': 'Starlight Dance Center'}, {'Name': 'Marie Wildey School of Dance'}, {'Name': 'Dance Academy of North Jersey'}, {'Name': 'Kaye Lynn Dance Studio I'}, {'Name': 'DDA Dance Academy'}, {'Name': 'Uptown Dance Studio'}, {'Name': 'DanceWorks Studios'}, {'Name': 'The Kennedy Dancers'}, {'Name': 'bboy bgirl lifestyle'}, {'Name': 'Dance Theatre of New Jersey'}, {'Name': 'Next Step Broadway'}, {'Name': 'Performing Arts Workshop'}]


In [4]:
Data = []
geolocator = Nominatim()
pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    print(prePage)
    startPage = "start="+ str(prePage) +"0"
    print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["Name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["Type"] = "Dance Classes"
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["Address"] = orgAddress[1]
                dataframe["City"] = orgAddress[2]
                dataframe["State"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["Zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["Address"] = orgAddress[lenAddress-3]
                dataframe["City"] = orgAddress[lenAddress-2]
                dataframe["State"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["Zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["Address"] = orgAddress[lenAddress-2]
                dataframe["City"] = orgAddress[lenAddress-1]
                dataframe["State"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["Zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
                print(orgAddress)
            try:
                locForLatLong = dataframe["Address"] + dataframe["City"]
                print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                print(location.address)
                print((location.latitude,location.longitude))
                dataframe["Location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)
df1=pd.DataFrame(top_result)



1
/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=Z_PjXYqRGKqvggfduIbgDg&start=20&sa=N&ved=0ahUKEwjK4LGG-JTmAhWql-AKHV2cAewQ8tMDCPYC
2
start=20
start=40
https://www.google.com/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=Z_PjXYqRGKqvggfduIbgDg&start=40&sa=N&ved=0ahUKEwjK4LGG-JTmAhWql-AKHV2cAewQ8tMDCPYC


C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


20
All+Children's+Art+Center+of+NJ,+15+John+St,+Bound+Brook,+NJ+08805
["All Children's Art Center of NJ", ' 15 John St', ' Bound Brook', ' NJ 08805']
 15 John St Bound Brook
15, John Street, Bound Brook, Somerset County, New Jersey, 08805, United States
(40.5618427634078, -74.5334232548041)
Newark+School+of+the+Arts,+89+Lincoln+Park,+Newark,+NJ+07102
['Newark School of the Arts', ' 89 Lincoln Park', ' Newark', ' NJ 07102']
 89 Lincoln Park Newark
89, Lincoln Park, Teachers Village, Newark, Essex County, New Jersey, 07102, United States
(40.7256942631579, -74.1794780526316)
Monroe+Street+Movement+Space,+720+Monroe+St,+Hoboken,+NJ+07030
['Monroe Street Movement Space', ' 720 Monroe St', ' Hoboken', ' NJ 07030']
 720 Monroe St Hoboken
720, Monroe Street, Hoboken, Hudson County, New Jersey, 07030, United States of America
(40.7466675918367, -74.0378767142857)
Dance+Starz+Academy+of+NJ,+496+Inman+Ave,+Rahway,+NJ+07065
['Dance Starz Academy of NJ', ' 496 Inman Ave', ' Rahway', ' NJ 07065']
 

618, Ridge Road, North Arlington, Bergen County, New Jersey, 07071, United States of America
(40.8021511, -74.1231799)
Dance+World+Academy,+334+Lakeview+Ave,+Clifton,+NJ+07011
['Dance World Academy', ' 334 Lakeview Ave', ' Clifton', ' NJ 07011']
 334 Lakeview Ave Clifton
334, Lakeview Avenue, Clifton, Passaic County, New Jersey, 07011, United States
(40.8865097142857, -74.1386048571429)
The+Dance+Connection,+295+US-46,+Rockaway,+NJ+07866
['The Dance Connection', ' 295 US-46', ' Rockaway', ' NJ 07866']
 295 US-46 Rockaway
Retro Fitness, 295, US 46, Rockaway, Morris County, New Jersey, 07866, United States of America
(40.891547, -74.522183)
Dance+World+Academy,+226+Main+Ave,+Passaic,+NJ+07055
['Dance World Academy', ' 226 Main Ave', ' Passaic', ' NJ 07055']
 226 Main Ave Passaic
226, Main Avenue, Passaic, Passaic County, New Jersey, 07055, United States
(40.8466307, -74.12665)
Jersey+City+Ballet,+189+Brunswick+St+2nd+Floor,+Jersey+City,+NJ+07302
['Jersey City Ballet', ' 189 Brunswick St 

Masters Institute of Music, 168, Franklin Turnpike, Waldwick, Bergen County, New Jersey, 07463, United States of America
(41.0204359, -74.1239628)
8
start=80
start=100
https://www.google.com/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=Z_PjXYqRGKqvggfduIbgDg&start=100&sa=N&ved=0ahUKEwjK4LGG-JTmAhWql-AKHV2cAewQ8tMDCPYC
20
Academy+of+Dance+%26+Performing,+1929+Morris+Ave,+Union,+NJ+07083
['Academy of Dance %26 Performing', ' 1929 Morris Ave', ' Union', ' NJ 07083']
 1929 Morris Ave Union
1929, Morris Avenue, Union, Union County, New Jersey, 07083, United States
(40.695712, -74.266432)
Dance+Art+Creative+Center,+519+Cedar+Ln,+Teaneck,+NJ+07666
['Dance Art Creative Center', ' 519 Cedar Ln', ' Teaneck', ' NJ 07666']
 519 Cedar Ln Teaneck
519, Cedar Lane, Teaneck Township, Bergen County, New Jersey, 07666, United States of America
(40.8896964375, -74.0235449375)
American+Dance+Studios+%2FSummer+Camp%2FEdison%2FPiscataway%2FMetuchen%2FNewBrunswickNJ,+8A+Case

4475, S. Clinton Avenue, South Plainfield, Middlesex County, New Jersey, 07080, United States of America
(40.562056, -74.4292479375)
Satrangi+School+of+Fusion,+1655+Oak+Tree+Rd,+Edison,+NJ+08820
['Satrangi School of Fusion', ' 1655 Oak Tree Rd', ' Edison', ' NJ 08820']
 1655 Oak Tree Rd Edison
1655, Oak Tree Road, Edison, Middlesex County, New Jersey, 08820, United States of America
(40.572816, -74.339406)
Quimbara+Dance+Studio,+605+Westfield+Ave,+Elizabeth,+NJ+07208
['Quimbara Dance Studio', ' 605 Westfield Ave', ' Elizabeth', ' NJ 07208']
 605 Westfield Ave Elizabeth
Westfield Avenue, Elizabeth, Union County, New Jersey, 07208, United States
(40.6680817, -74.2193436)
Broadway+Bound+Theatrical+Center,+231+Tontine+Ave,+Lyndhurst,+NJ+07071
['Broadway Bound Theatrical Center', ' 231 Tontine Ave', ' Lyndhurst', ' NJ 07071']
 231 Tontine Ave Lyndhurst
231, Tontine Avenue, Lyndhurst, Bergen County, New Jersey, 07071, United States
(40.815344902439, -74.1262673414634)
The+Academy+of+Dance+Ar

Cant Find the address
The+Star+Maker+School,+46+Old+Hwy+22,+Clinton,+NJ+08809
['The Star Maker School', ' 46 Old Hwy 22', ' Clinton', ' NJ 08809']
 46 Old Hwy 22 Clinton
46, Old Highway 22, Clinton, Hunterdon County, New Jersey, 08809, United States
(40.6369091716717, -74.9086269028621)
Ballet+Central+New+Jersey,+4+Tennis+Ct,+Hamilton+Township,+NJ+08619
['Ballet Central New Jersey', ' 4 Tennis Ct', ' Hamilton Township', ' NJ 08619']
 4 Tennis Ct Hamilton Township
4, Tennis Court, Hamilton Township, Mercer County, New Jersey, 08619, United States
(40.2650260010497, -74.6792350895572)
14
start=140
start=160
https://www.google.com/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=Z_PjXYqRGKqvggfduIbgDg&start=160&sa=N&ved=0ahUKEwjK4LGG-JTmAhWql-AKHV2cAewQ8tMDCPYC
20
Ambition+Dance+Academy,+41+Bergenline+Ave,+Westwood,+NJ+07675
['Ambition Dance Academy', ' 41 Bergenline Ave', ' Westwood', ' NJ 07675']
 41 Bergenline Ave Westwood
41, Bergenline Avenue, Emerson, 

355, Eisenhower Parkway, Livingston, Essex County, New Jersey, 07039, United States
(40.7967800408163, -74.3412397346939)
Ajna+Dance+Company,+18+Bleecker+St,+New+York,+NY+10012
['Ajna Dance Company', ' 18 Bleecker St', ' New York', ' NY 10012']
 18 Bleecker St New York
Sheen Center, 18, Bleecker Street, NoHo, Manhattan Community Board 2, Manhattan, New York County, New York, 10012, United States of America
(40.7254212, -73.9932418)
BollyMoves+Dance+Academy,+26+Fela+Dr,+Parlin,+NJ+08859
['BollyMoves Dance Academy', ' 26 Fela Dr', ' Parlin', ' NJ 08859']
 26 Fela Dr Parlin
Cant Find the address
Rising+Stars+School+of+Dance,+337+U.S.+9+%234,+Englishtown,+NJ+07726
['Rising Stars School of Dance', ' 337 U.S. 9 %234', ' Englishtown', ' NJ 07726']
 337 U.S. 9 %234 Englishtown
Cant Find the address
Colts+Neck+Dance+%26+Performing,+410+NJ-34,+Colts+Neck,+NJ+07722
['Colts Neck Dance %26 Performing', ' 410 NJ-34', ' Colts Neck', ' NJ 07722']
 410 NJ-34 Colts Neck
JCT Route 34/537, Colts Neck Town

Atlantic City Boulevard, Toms River, Ocean County, New Jersey, 08722, United States
(39.9516366, -74.1987784)
Ventura+Dance+Center,+540+Durie+Ave,+Closter,+NJ+07624
['Ventura Dance Center', ' 540 Durie Ave', ' Closter', ' NJ 07624']
 540 Durie Ave Closter
540, Durie Avenue, Closter, Bergen County, New Jersey, 07624, United States of America
(40.9721136190476, -73.9621966190476)


In [5]:
#df1.to_excel("outputDance.xlsx") 
df1


,Name,Type,phoneNo,Address,City,State,Zipcode,Location
0,All Children's Art Center of NJ,Dance Classes,,15 John St,Bound Brook,NJ,08805,"[40.5618427634078, -74.5334232548041]"
1,Newark School of the Arts,Dance Classes,,89 Lincoln Park,Newark,NJ,07102,"[40.7256942631579, -74.1794780526316]"
2,Monroe Street Movement Space,Dance Classes,,720 Monroe St,Hoboken,NJ,07030,"[40.7466675918367, -74.0378767142857]"
3,Dance Starz Academy of NJ,Dance Classes,,496 Inman Ave,Rahway,NJ,07065,"[40.6001789, -74.3113377]"
4,Dynamic Dance Academy,Dance Classes,,546 Bloomfield Ave,Verona,NJ,07044,"[40.8345572, -74.2564094]"
...,...,...,...,...,...,...,...,...
175,Dance Corner Inc,Dance Classes,430,335 Princeton Hightstown Rd,West Windsor Township,NJ,08550,"[40.2866978003687, -74.5657156296177]"
176,Dance for Joy,Dance Classes,7322235336,1025 NJ-70 Suite 2,Brielle,NJ,08730,NaN
177,Lambarri Dance Arts,Dance Classes,330,1070 NJ-34 %23250,Matawan,NJ,07747,"[40.3998465, -74.2290962]"
178,The Dance Centre,Dance Classes,,316 Atlantic City Blvd,Toms River,NJ,08757,"[39.9516366, -74.1987784]"


## Google Music

In [6]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&tbm=lcl&sxsrf=ACYBGNTdFaDd4jAeC6lZKeILHtxz3pMx4A%3A1574389394556&ei=kkbXXanHIeSV_Qbz-a_wAg&q=kids+music+classes+new+jersey&oq=kids+music+classes+new+jersey&gs_l=psy-ab.12...0.0.0.163915.0.0.0.0.0.0.0.0..0.0....0...1c..64.psy-ab..0.0.0....0.51R-_vCUfl4#rlfi=hd:;si:;mv:[[41.0328425,-73.9399237],[40.1005597,-74.3127616]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!1m4!1u16!2m2!16m1!1e1!1m4!1u16!2m2!16m1!1e2!2m1!1e2!2m1!1e3!2m1!1e16!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["Name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'Name': 'West Essex Music Together'}, {'Name': 'Montclair Music Studio'}, {'Name': 'The Institute of Music for Children'}, {'Name': 'Little Rockers - Cool Kids Music Classes'}, {'Name': 'Musical Munchkins of New Jersey'}, {'Name': 'Little Rockers - cool kids music classes & parties'}, {'Name': 'Little Rockers - cool kids music classes & parties'}, {'Name': 'Music Together of Bergen County'}, {'Name': 'Rockness Music'}, {'Name': 'Lindeblad School of Music'}, {'Name': 'Joy Kids Music & Art (Joy Performing Art Center)'}, {'Name': 'Little Notes Music School'}, {'Name': 'Music Together of Montclair'}, {'Name': 'ShreeRaga Carnatic Vocal Music School'}, {'Name': "Sharon's Studio of Dance & Music"}, {'Name': 'The Extension Division at Cali School of Music'}, {'Name': 'West Essex Music Together'}, {'Name': 'Rockness Music'}, {'Name': 'Do Re Mi School of Music and the Arts'}, {'Name': 'A Sound Start'}, {'Name': 'Edison School of Music and Dance'}, {'Name': 'New Jersey Conservatory'}]


In [7]:
Data = []
geolocator = Nominatim()
pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    print(prePage)
    startPage = "start="+ str(prePage) +"0"
    print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["Name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["Type"] = "Music Classes"
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["Address"] = orgAddress[1]
                dataframe["City"] = orgAddress[2]
                dataframe["State"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["Zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["Address"] = orgAddress[lenAddress-3]
                dataframe["City"] = orgAddress[lenAddress-2]
                dataframe["State"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["Zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["Address"] = orgAddress[lenAddress-2]
                dataframe["City"] = orgAddress[lenAddress-1]
                dataframe["State"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["Zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
                print(orgAddress)
            try:
                locForLatLong = dataframe["Address"] + dataframe["City"]
                print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                print(location.address)
                print((location.latitude,location.longitude))
                dataframe["Location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)
df2=pd.DataFrame(top_result)



1
/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=y_PjXf3DCIqc_QaUlqaAAg&start=20&sa=N&ved=0ahUKEwi91fm1-JTmAhUKTt8KHRSLCSAQ8tMDCM0C
2
start=20
start=40
https://www.google.com/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=y_PjXf3DCIqc_QaUlqaAAg&start=40&sa=N&ved=0ahUKEwi91fm1-JTmAhUKTt8KHRSLCSAQ8tMDCM0C


C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


20
Newark+School+of+the+Arts,+89+Lincoln+Park,+Newark,+NJ+07102
['Newark School of the Arts', ' 89 Lincoln Park', ' Newark', ' NJ 07102']
 89 Lincoln Park Newark
89, Lincoln Park, Teachers Village, Newark, Essex County, New Jersey, 07102, United States
(40.7256942631579, -74.1794780526316)
Music+Therapy+for+all+Ages,+568+Downer+St,+Westfield,+NJ+07090
['Music Therapy for all Ages', ' 568 Downer St', ' Westfield', ' NJ 07090']
 568 Downer St Westfield
568, Downer Street, Westfield, Union County, New Jersey, 07090, United States
(40.6450615714286, -74.3567345510204)
Kids'+Music+Round,+14+Gulick+Ln,+Plainsboro+Township,+NJ+08536
["Kids' Music Round", ' 14 Gulick Ln', ' Plainsboro Township', ' NJ 08536']
 14 Gulick Ln Plainsboro Township
14, Gulick Lane, Plainsboro Township, Middlesex County, New Jersey, 08536, United States of America
(40.3544950378446, -74.5807505891285)
Jazz+House+Kids,+347+Bloomfield+Ave,+Montclair,+NJ+07042
['Jazz House Kids', ' 347 Bloomfield Ave', ' Montclair', ' NJ

NJ 35, Red Bank, Monmouth County, New Jersey, 07701, United States
(40.3572926, -74.0773583)
Fort+Lee+School+of+Music,+2175+Lemoine+Ave+Suite+503,+Fort+Lee,+NJ+07024
['Fort Lee School of Music', ' 2175 Lemoine Ave Suite 503', ' Fort Lee', ' NJ 07024']
 2175 Lemoine Ave Suite 503 Fort Lee
Cant Find the address
Cranford+School+of+Music+and+Dance,+777+Walnut+Ave,+Cranford,+NJ+07016
['Cranford School of Music and Dance', ' 777 Walnut Ave', ' Cranford', ' NJ 07016']
 777 Walnut Ave Cranford
777, Walnut Avenue, Cranford, Union County, New Jersey, 07016, United States of America
(40.6356734615385, -74.3038773461538)
Yard+School+of+Art,+3+S+Mountain+Ave,+Montclair,+NJ+07042
['Yard School of Art', ' 3 S Mountain Ave', ' Montclair', ' NJ 07042']
 3 S Mountain Ave Montclair
Montclair Art Museum, 3, South Mountain Avenue, Montclair, Essex County, New Jersey, 07042, United States
(40.81860135, -74.2242380490856)
Taalim+School+of+Indian+Music,+15+Chokeberry+Dr,+Edison,+NJ+08837
['Taalim School of In

20
Eastside+Westside+Music+Together,+428+E+75th+St,+New+York,+NY+10021
['Eastside Westside Music Together', ' 428 E 75th St', ' New York', ' NY 10021']
 428 E 75th St New York
428, East 75th Street, Upper East Side, Manhattan Community Board 8, Manhattan, New York County, New York, 10021, United States of America
(40.76910705, -73.953490605997)
Kidville,+1202+Shipyard+Ln,+Hoboken,+NJ+07030
['Kidville', ' 1202 Shipyard Ln', ' Hoboken', ' NJ 07030']
 1202 Shipyard Ln Hoboken
Cant Find the address
A+Hum+Music+Waldwick,+136+Hopper+Ave,+Waldwick,+NJ+07463
['A Hum Music Waldwick', ' 136 Hopper Ave', ' Waldwick', ' NJ 07463']
 136 Hopper Ave Waldwick
136, Hopper Avenue, Waldwick, Bergen County, New Jersey, 07463, United States of America
(41.0194609, -74.12665595)
Kidville,+516+Valley+Rd,+Montclair,+NJ+07043
['Kidville', ' 516 Valley Rd', ' Montclair', ' NJ 07043']
 516 Valley Rd Montclair
516, Valley Road, Montclair, Essex County, New Jersey, 07043, United States of America
(40.837221, -74.2

394, Union Boulevard, Totowa, Passaic County, New Jersey, 07512, United States of America
(40.902455, -74.2141283333333)
Tanvi+Art+Class,+Edison,+NJ+08820
['Tanvi Art Class', ' Edison', ' NJ 08820']
['Tanvi Art Class', ' Edison', ' NJ 08820']
['Tanvi Art Class', ' Edison', ' NJ 08820']
 Edison NJ 08820
Edison, Middlesex County, New Jersey, United States of America
(40.5382375, -74.3945173450877)
Paint+Mix+NJ,+30+Park+St,+Montclair,+NJ+07042
['Paint Mix NJ', ' 30 Park St', ' Montclair', ' NJ 07042']
 30 Park St Montclair
30, Park Street, Montclair, Essex County, New Jersey, 07043, United States of America
(40.8310325774648, -74.2061450140845)
Gymboree+Play+%26+Music,+Pine+Brook,+321+Changebridge+Rd,+Pine+Brook,+NJ+07058
['Gymboree Play %26 Music', ' Pine Brook', ' 321 Changebridge Rd', ' Pine Brook', ' NJ 07058']
['', 'NJ', '07058']
 321 Changebridge Rd Pine Brook
Changebridge Road, Montville Township, Morris County, New Jersey, 07058, United States of America
(40.866879, -74.355021)
Ki

712, NJ 18, Tanners Corner, East Brunswick Township, Middlesex County, New Jersey, 08816, United States of America
(40.4290995, -74.3839451666667)
Idol+Time+Vocal+Academy,+2140+NJ-88,+Brick+Township,+NJ+08724
['Idol Time Vocal Academy', ' 2140 NJ-88', ' Brick Township', ' NJ 08724']
 2140 NJ-88 Brick Township
2140, NJ 88, Point Pleasant, Ocean County, New Jersey, 08724, United States of America
(40.0768027142857, -74.0997482857143)
14
start=140
start=160
https://www.google.com/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=y_PjXf3DCIqc_QaUlqaAAg&start=160&sa=N&ved=0ahUKEwi91fm1-JTmAhUKTt8KHRSLCSAQ8tMDCM0C
20
Idol+Time+Vocal+Academy,+2140+NJ-88,+Brick+Township,+NJ+08724
['Idol Time Vocal Academy', ' 2140 NJ-88', ' Brick Township', ' NJ 08724']
 2140 NJ-88 Brick Township
2140, NJ 88, Point Pleasant, Ocean County, New Jersey, 08724, United States of America
(40.0768027142857, -74.0997482857143)
Lumberton+School+of+Music,+1+Rockland+Terrace,+Lumberton,+NJ+0

0


In [8]:
#df.to_excel("outputMusic.xlsx") 
df2

,Name,Type,phoneNo,Address,City,State,Zipcode,Location
0,Newark School of the Arts,Music Classes,,89 Lincoln Park,Newark,NJ,07102,"[40.7256942631579, -74.1794780526316]"
1,Music Therapy for all Ages,Music Classes,8662793049,568 Downer St,Westfield,NJ,07090,"[40.6450615714286, -74.3567345510204]"
2,Kids' Music Round,Music Classes,6099369069,14 Gulick Ln,Plainsboro Township,NJ,08536,"[40.3544950378446, -74.5807505891285]"
3,Kids' MusicRound,Music Classes,6093330100,NaN,NaN,NaN,NaN,NaN
4,Jazz House Kids,Music Classes,,347 Bloomfield Ave,Montclair,NJ,07042,"[40.8100079, -74.2129392]"
...,...,...,...,...,...,...,...,...
140,Paint Fun Studios,Music Classes,10,3231 NJ-27,Franklin Park,NJ,08823,"[40.4339564751802, -74.5460070547637]"
141,Children's Home Montessori School,Music Classes,8,777 Anderson Ave,Cliffside Park,NJ,07010,"[40.831473, -73.985152]"
142,Dance It Up,Music Classes,6095810808,1800 NJ-33,Hamilton Square,NJ,08690,"[40.220972, -74.6357047]"
143,Kidville,Music Classes,2019831551,53 Vervalen St,Closter,NJ,07624,"[40.9704506177326, -73.9582628723433]"


## Google Sports

In [9]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=05bdXaEyoqX9BoGbkuAK&q=kids+sports+classes+new+jersey&oq=kids+sports+classes+new+jersey&gs_l=psy-ab.12...13265.16654.0.22362.7.7.0.0.0.0.83.504.7.7.0....0...1c.1.64.psy-ab..0.0.0....0.EEUl6U13Yy0#rlfi=hd:;si:;mv:[[41.1122716,-73.93472659999999],[40.3136717,-74.6914486]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!2m1!1e3!3sIAE,lf:1,lf_ui:3'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["Name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

21
[{'Name': 'Life Time Athletic'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'The GYM For Kids'}, {'Name': 'The Little Gym of Montclair'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}, {'Name': 'New York Sports Clubs'}]


In [10]:
Data = []
geolocator = Nominatim()
geolocator = Nominatim()
pageData = soup.find_all(class_='std')
#print( soup.find("div", class_="std")["aria-label"] )

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12] 
for x in (list1):
    prePage = x
    print(prePage)
    startPage = "start="+ str(prePage) +"0"
    print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["Name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["Type"] = "Sports Classes"
        
        PhoneData = tr.find('span').text.split(' ')
        print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["Address"] = orgAddress[1]
                dataframe["City"] = orgAddress[2]
                dataframe["State"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["Zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["Address"] = orgAddress[lenAddress-3]
                dataframe["City"] = orgAddress[lenAddress-2]
                dataframe["State"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["Zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["Address"] = orgAddress[lenAddress-2]
                dataframe["City"] = orgAddress[lenAddress-1]
                dataframe["State"] = ZipCode[1]
                if len(ZipCode) > 2:
                    dataframe["Zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                print(orgAddress)
                    
            try:
                locForLatLong = dataframe["Address"] + dataframe["City"]
                print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                print(location.address)
                print((location.latitude,location.longitude))
                dataframe["Location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
        top_result.append(dataframe)

#print(top_result)
df3=pd.DataFrame(top_result)



1
/search?q=kids+sports+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=HvTjXZDOIOSGggeK15uQDA&start=20&sa=N&ved=0ahUKEwiQ1dvd-JTmAhVkg-AKHYrrBsIQ8tMDCKwC
2
start=20
start=40
https://www.google.com/search?q=kids+sports+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=HvTjXZDOIOSGggeK15uQDA&start=40&sa=N&ved=0ahUKEwiQ1dvd-JTmAhVkg-AKHYrrBsIQ8tMDCKwC


C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  
C:\Users\13157\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the defaul

20
['Martial', 'arts', 'school16.5', 'mi', '·', 'Jersey', 'City,', 'NJ(201)', '222-8996Closed', '⋅', 'Opens', '12PM', 'MonTheir', 'website', 'mentions', 'children', 'and', 'classes']
Family+Fitness+Karate+%26+Kickboxing+-+Central+Ave,+419+Central+Ave,+Jersey+City,+NJ+07307
['Family Fitness Karate %26 Kickboxing - Central Ave', ' 419 Central Ave', ' Jersey City', ' NJ 07307']
 419 Central Ave Jersey City
419, Central Avenue, Jersey City, Hudson County, New Jersey, 07307, United States of America
(40.7490070909091, -74.0474060909091)
['Gymnastics', 'center32.0', 'mi', '·', 'Matawan,', 'NJ(732)', '772-0060']
The+Little+Gym+of+Aberdeen,+1121+NJ-34+Suite+O,+Matawan,+NJ+07747
['The Little Gym of Aberdeen', ' 1121 NJ-34 Suite O', ' Matawan', ' NJ 07747']
 1121 NJ-34 Suite O Matawan
Cant Find the address
['Gym71.1', 'mi', '·', 'Maple', 'Shade', 'Township,', 'NJ(856)', '234-6445Closed', '⋅', 'Opens', '10AM', 'MonTheir', 'website', 'mentions', 'sport', 'classes']
LittleSport,+120+E+Kings+Hwy+%23

Sheraton Parsippany Hotel, 199, Smith Road, Parsippany-Troy Hills, Morris County, New Jersey, 07054, United States
(40.8616728, -74.4124167547163)
['Yoga', 'studio5.1', 'mi', '·', 'Montville,', 'NJ(973)', '495-0334']
Nirvana+Yoga+and+Wellness+Center,+112+Main+Rd,+Montville,+NJ+07045
['Nirvana Yoga and Wellness Center', ' 112 Main Rd', ' Montville', ' NJ 07045']
 112 Main Rd Montville
112, Main Road, Boonton, Morris County, New Jersey, 07045, United States of America
(40.9155099288288, -74.3789250895736)
['Sports', 'complex19.7', 'mi', '·', 'Flanders,', 'NJ(973)', '584-9322Open', '⋅', 'Closes', '8PM']
Centercourt+Club+%26+Sports+-+Mt.+Olive,+184+Flanders+Netcong+Rd,+Flanders,+NJ+07836
['Centercourt Club %26 Sports - Mt. Olive', ' 184 Flanders Netcong Rd', ' Flanders', ' NJ 07836']
 184 Flanders Netcong Rd Flanders
Flanders Netcong Road, Mount Olive, Morris County, New Jersey, 07836, United States
(40.855718, -74.705549)
['Martial', 'arts', 'school7.6', 'mi', '·', 'City', 'of', 'Orange,'

25, Park Street, Montclair, Essex County, New Jersey, 07042, United States of America
(40.8159507142857, -74.2182045714286)
['Martial', 'arts', 'school3.1', 'mi', '·', 'Livingston,', 'NJ(973)', '422-1500Closed', '⋅', 'Opens', '7:30AM', 'Mon']
Integrated+Martial+Arts+%26+Fitness,+355+Eisenhower+Pkwy,+Livingston,+NJ+07039
['Integrated Martial Arts %26 Fitness', ' 355 Eisenhower Pkwy', ' Livingston', ' NJ 07039']
 355 Eisenhower Pkwy Livingston
355, Eisenhower Parkway, Livingston, Essex County, New Jersey, 07039, United States
(40.7967800408163, -74.3412397346939)
['Fencing', 'school9.9', 'mi', '·', 'Maplewood,', 'NJ(973)', '762-6363Closed', '⋅', 'Opens', '4PM', 'Mon']
New+Jersey+Fencing+Alliance,+50+Burnett+Ave,+Maplewood,+NJ+07040
['New Jersey Fencing Alliance', ' 50 Burnett Ave', ' Maplewood', ' NJ 07040']
 50 Burnett Ave Maplewood
50, Burnett Avenue, Maplewood, Essex County, New Jersey, 07040, United States of America
(40.720866, -74.2591200769231)
['Martial', 'arts', 'school8.1', 'mi

Cant Find the address
['Gymnastics', 'center', '·', 'Barclay', 'Farm', 'Shopping', 'Center73.9', 'mi', '·', 'Cherry', 'Hill,', 'NJ(856)', '857-0200Open', '⋅', 'Closes', '9PM']
My+Gym,+170+Barclay+Farms+Shopping+Center,+Cherry+Hill,+NJ+08034
['My Gym', ' 170 Barclay Farms Shopping Center', ' Cherry Hill', ' NJ 08034']
 170 Barclay Farms Shopping Center Cherry Hill
1409, Barclay Farms Shopping Centre, Cherry Hill, Camden County, New Jersey, 08034, United States
(39.910505, -74.9959876303868)
['Martial', 'arts', 'school9.7', 'mi', '·', 'Maplewood,', 'NJ(908)', '967-8132Closed', '⋅', 'Opens', '1:30PM']
Blue+Life+Karate+%26+Kickboxing+Centers,+1882+Springfield+Ave,+Maplewood,+NJ+07040
['Blue Life Karate %26 Kickboxing Centers', ' 1882 Springfield Ave', ' Maplewood', ' NJ 07040']
 1882 Springfield Ave Maplewood
1882, Springfield Avenue, Maplewood, Essex County, New Jersey, 07040, United States of America
(40.720718, -74.270127)
['Gym3.8', 'mi', '·', 'East', 'Hanover,', 'NJ(973)', '585-6008Cl

In [11]:
#df.to_excel("outputSports.xlsx") 
df3


,Name,Type,phoneNo,Address,City,State,Zipcode,Location
0,Family Fitness Karate & Kickboxing - Central Ave,Sports Classes,,419 Central Ave,Jersey City,NJ,07307,"[40.7490070909091, -74.0474060909091]"
1,The Little Gym of Aberdeen,Sports Classes,7327720060,1121 NJ-34 Suite O,Matawan,NJ,07747,NaN
2,LittleSport,Sports Classes,,120 E Kings Hwy %23403,Maple Shade Township,NJ,08052,"[39.9332787373737, -74.9894579090909]"
3,Kid Fit NJ,Sports Classes,8,2040 Springdale Rd,Cherry Hill,NJ,08003,"[39.26813625, -84.53199775]"
4,Madison Area YMCA,Sports Classes,,111 Kings Rd,Madison,NJ,07940,"[40.75362505, -74.412563970724]"
...,...,...,...,...,...,...,...,...
63,My Gym,Sports Classes,9,170 Barclay Farms Shopping Center,Cherry Hill,NJ,08034,"[39.910505, -74.9959876303868]"
64,Blue Life Karate & Kickboxing Centers,Sports Classes,130,1882 Springfield Ave,Maplewood,NJ,07040,"[40.720718, -74.270127]"
65,Precision Sports Performance,Sports Classes,8,50 Williams Pkwy Unit D,East Hanover,NJ,07936,NaN
66,Centercourt Tennis Center - Florham Park,Sports Classes,9738221900,299 Columbia Turnpike,Florham Park,NJ,07932,"[40.7904322832487, -74.4115379307443]"


## Merged all three dataframe into one DF

In [12]:

df = df1.append(df2, ignore_index=True)
df = df.append(df3, ignore_index=True)
df

,Name,Type,phoneNo,Address,City,State,Zipcode,Location
0,All Children's Art Center of NJ,Dance Classes,,15 John St,Bound Brook,NJ,08805,"[40.5618427634078, -74.5334232548041]"
1,Newark School of the Arts,Dance Classes,,89 Lincoln Park,Newark,NJ,07102,"[40.7256942631579, -74.1794780526316]"
2,Monroe Street Movement Space,Dance Classes,,720 Monroe St,Hoboken,NJ,07030,"[40.7466675918367, -74.0378767142857]"
3,Dance Starz Academy of NJ,Dance Classes,,496 Inman Ave,Rahway,NJ,07065,"[40.6001789, -74.3113377]"
4,Dynamic Dance Academy,Dance Classes,,546 Bloomfield Ave,Verona,NJ,07044,"[40.8345572, -74.2564094]"
...,...,...,...,...,...,...,...,...
388,My Gym,Sports Classes,9,170 Barclay Farms Shopping Center,Cherry Hill,NJ,08034,"[39.910505, -74.9959876303868]"
389,Blue Life Karate & Kickboxing Centers,Sports Classes,130,1882 Springfield Ave,Maplewood,NJ,07040,"[40.720718, -74.270127]"
390,Precision Sports Performance,Sports Classes,8,50 Williams Pkwy Unit D,East Hanover,NJ,07936,NaN
391,Centercourt Tennis Center - Florham Park,Sports Classes,9738221900,299 Columbia Turnpike,Florham Park,NJ,07932,"[40.7904322832487, -74.4115379307443]"


In [13]:
df.to_excel("outputKidsActivities.xlsx") 

In [14]:
df=df.reset_index()
df.head()

,index,Name,Type,phoneNo,Address,City,State,Zipcode,Location
0,0,All Children's Art Center of NJ,Dance Classes,,15 John St,Bound Brook,NJ,08805,"[40.5618427634078, -74.5334232548041]"
1,1,Newark School of the Arts,Dance Classes,,89 Lincoln Park,Newark,NJ,07102,"[40.7256942631579, -74.1794780526316]"
2,2,Monroe Street Movement Space,Dance Classes,,720 Monroe St,Hoboken,NJ,07030,"[40.7466675918367, -74.0378767142857]"
3,3,Dance Starz Academy of NJ,Dance Classes,,496 Inman Ave,Rahway,NJ,07065,"[40.6001789, -74.3113377]"
4,4,Dynamic Dance Academy,Dance Classes,,546 Bloomfield Ave,Verona,NJ,07044,"[40.8345572, -74.2564094]"


In [26]:
#df = df.rename(columns={'index':'_id'})
df = df.drop(df.index[0])
#df = df.drop(columns=['Reviews'])
#df = df.drop(columns=['_id'])
df.head()

,_id,Name,Type,phoneNo,Address,City,State,Zipcode,Location
1,1,Newark School of the Arts,Dance Classes,,89 Lincoln Park,Newark,NJ,07102,"[40.7256942631579, -74.1794780526316]"
2,2,Monroe Street Movement Space,Dance Classes,,720 Monroe St,Hoboken,NJ,07030,"[40.7466675918367, -74.0378767142857]"
3,3,Dance Starz Academy of NJ,Dance Classes,,496 Inman Ave,Rahway,NJ,07065,"[40.6001789, -74.3113377]"
4,4,Dynamic Dance Academy,Dance Classes,,546 Bloomfield Ave,Verona,NJ,07044,"[40.8345572, -74.2564094]"
5,5,Broadway Bound Dance Center Inc.,Dance Classes,,99 W Madison Ave,Dumont,NJ,07628,"[40.94016, -73.9985329090909]"


In [27]:
split_data = df.to_dict('records')
split_data

[{'_id': 1,
  'Name': 'Newark School of the Arts',
  'Type': 'Dance Classes',
  'phoneNo': '',
  'Address': ' 89 Lincoln Park',
  'City': ' Newark',
  'State': 'NJ',
  'Zipcode': '07102',
  'Location': [40.7256942631579, -74.1794780526316]},
 {'_id': 2,
  'Name': 'Monroe Street Movement Space',
  'Type': 'Dance Classes',
  'phoneNo': '',
  'Address': ' 720 Monroe St',
  'City': ' Hoboken',
  'State': 'NJ',
  'Zipcode': '07030',
  'Location': [40.7466675918367, -74.0378767142857]},
 {'_id': 3,
  'Name': 'Dance Starz Academy of NJ',
  'Type': 'Dance Classes',
  'phoneNo': '',
  'Address': ' 496 Inman Ave',
  'City': ' Rahway',
  'State': 'NJ',
  'Zipcode': '07065',
  'Location': [40.6001789, -74.3113377]},
 {'_id': 4,
  'Name': 'Dynamic Dance Academy',
  'Type': 'Dance Classes',
  'phoneNo': '',
  'Address': ' 546 Bloomfield Ave',
  'City': ' Verona',
  'State': 'NJ',
  'Zipcode': '07044',
  'Location': [40.8345572, -74.2564094]},
 {'_id': 5,
  'Name': 'Broadway Bound Dance Center Inc.',

In [28]:
df.dtypes

_id          int64
Name        object
Type        object
phoneNo     object
Address     object
City        object
State       object
Zipcode     object
Location    object
dtype: object

In [30]:
# Set up the MongoDB connection through pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
# Create DB
db = myclient["KidsRidedb3"]
# Create collection and insert all the data into the MongoDB
mycol = db["activityData1"]
mycol.insert_many(split_data)